In [1]:
# All the imports go here
import cv2
import numpy as np
import mediapipe as mp
from collections import deque

In [2]:
# Giving different arrays to handle colour points of different colour
rpoints = [deque(maxlen=1020)]
bpoints = [deque(maxlen=1020)]
gpoints = [deque(maxlen=1020)]
ypoints = [deque(maxlen=1020)]

In [3]:
# These indexes will be used to mark the points in particular arrays of specific colour
red_index = 0
blue_index = 0
green_index = 0
yellow_index = 0

In [4]:
#The kernel to be used for dilation purpose 
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
colorIndex = 0

In [5]:
# Here is code for Canvas setup
paintWindow = np.zeros((471,636,3)) + 255
paintWindow = cv2.rectangle(paintWindow, (40,1), (140,65), (0,0,0), 2)
paintWindow = cv2.rectangle(paintWindow, (160,1), (255,65), (255,0,0), 2)
paintWindow = cv2.rectangle(paintWindow, (275,1), (370,65), (0,255,0), 2)
paintWindow = cv2.rectangle(paintWindow, (390,1), (485,65), (0,0,255), 2)
paintWindow = cv2.rectangle(paintWindow, (505,1), (600,65), (0,255,255), 2)

cv2.putText(paintWindow, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)

In [6]:
# initialize mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

I0000 00:00:1737120302.145539  286401 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-23.0.22), renderer: Intel(R) Iris(TM) Plus Graphics 645
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1737120302.205102  287432 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737120302.243952  287432 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [7]:
# Initialize the webcam
cap = cv2.VideoCapture(0)
ret = True
while ret:
    # Read each frame from the webcam
    ret, frame = cap.read()
    x, y, c = frame.shape
    
    #Flip the frame vertically
    frame = cv2.flip(frame,1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    frame = cv2.rectangle(frame, (40,1), (140,65), (0,0,0), 2)
    frame = cv2.rectangle(frame, (160,1), (255,65), (255,0,0), 2)
    frame = cv2.rectangle(frame, (275,1), (370,65), (0,255,0), 2)
    frame = cv2.rectangle(frame, (390,1), (485,65), (0,0,255), 2)
    frame = cv2.rectangle(frame, (505,1), (600,65), (0,255,255), 2)
    
    cv2.putText(frame, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    #frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # Get hand landmark prediction
    result = hands.process(framergb)

    # post process the result
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                lmx = int(lm.x * 640)
                lmy = int(lm.y * 480)

                landmarks.append([lmx, lmy])


    # Drawing landmarks on frames
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)
        fore_finger = (landmarks[8][0],landmarks[8][1])
        center = fore_finger
        thumb = (landmarks[4][0],landmarks[4][1])
        cv2.circle(frame, center,3,(0,255,0),-1)
        print(center[1]-thumb[1])
        if (thumb[1]-center[1]<30):
            bpoints.append(deque(maxlen=512))
            blue_index += 1
            gpoints.append(deque(maxlen=512))
            green_index += 1
            rpoints.append(deque(maxlen=512))
            red_index += 1
            ypoints.append(deque(maxlen=512))
            yellow_index += 1
         
        elif center[1] <= 65:
             # Clear Button
            if 40 <= center[0] <= 140:
                bpoints = [deque(maxlen=512)]
                gpoints = [deque(maxlen=512)]
                rpoints = [deque(maxlen=512)]
                ypoints = [deque(maxlen=512)]

                blue_index = 0
                green_index = 0
                red_index = 0
                yellow_index = 0

                paintWindow[67:,:,:] = 255
            elif 160 <= center[0] <= 255:
                     # Blue
                    colorIndex = 0 
            elif 275 <= center[0] <= 370:
                    # Green
                    colorIndex = 1 
            elif 390 <= center[0] <= 485:
                     # Red
                    colorIndex = 2 
            elif 505 <= center[0] <= 600:
                     # Yellow
                    colorIndex = 3 
        else :
            if colorIndex == 0:
                bpoints[blue_index].appendleft(center)
            elif colorIndex == 1:
                gpoints[green_index].appendleft(center)
            elif colorIndex == 2:
                rpoints[red_index].appendleft(center)
            elif colorIndex == 3:
                ypoints[yellow_index].appendleft(center)
    # Append the next deques when nothing is detected to avois messing up
    else:
        bpoints.append(deque(maxlen=512))
        blue_index += 1
        gpoints.append(deque(maxlen=512))
        green_index += 1
        rpoints.append(deque(maxlen=512))
        red_index += 1
        ypoints.append(deque(maxlen=512))
        yellow_index += 1

    # Draw lines of all the colors on the canvas and frame
    points = [bpoints, gpoints, rpoints, ypoints]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    cv2.imshow("Output", frame) 
    cv2.imshow("Paint", paintWindow)

    if cv2.waitKey(1) == ord('q'):
        break
# release the webcam and destroy all active windows# release the webcam and destroy all active windows
cap.release()
cv2.destroyAllWindows()

2025-01-17 17:25:05.593 Python[30800:286401] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2025-01-17 17:25:07.412 Python[30800:286401] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-17 17:25:07.412 Python[30800:286401] +[IMKInputSession subclass]: chose IMKInputSession_Modern
W0000 00:00:1737120314.909993  287431 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


-46
-21
-20
-85
-97
-103
-107
-114
-120
-124
-123
-131
-132
-136
-139
-130
-128
-120
-115
-110
-100
-99
-99
-97
-100
-102
-105
-109
-113
-112
-111
-110
-112
-111
-111
-111
-110
-111
-112
-111
-112
-111
-111
-111
-110
-110
-110
-110
-110
-110
-109
-106
-103
-102
-100
-98
-98
-100
-98
-105
-110
-113
-112
-112
-113
-117
-123
-121
-122
-121
-123
-124
-122
-118
-115
-113
-113
-113
-114
-114
-118
-117
-116
-113
-114
-113
-111
-113
-118
-117
-120
-125
-125
-127
-121
-123
-124
-124
-123
-117
-114
-116
-116
-117
-116
-114
-110
-111
-110
-111
-110
-111
-115
-120
-123
-126
-124
-125
-125
-126
-124
-121
-120
-117
-106
-103
-102
-107
-113
-114
-115
-117
-119
-125
-125
-128
-129
-126
-125
-125
-125
-121
-117
-117
-119
-119
-121
-121
-124
-123
-125
-128
-129
-126
-126
-130
-134
-135
-137
-132
-128
-127
-123
-119
-119
-124
-117
-111
-109
-101
27
34
45
-39
-76
-97
-107
-116
-110
-116
-116
-119
-118
-115
-114
-116
-117
-117
-113
-111
-108
7
-100
-106
-114
-116
-115
-116
-117
-90
-30
-4
15
14
11
-27
-32


KeyboardInterrupt: 